## Opens a basemap in image location and prints list of coordinates selected by user

## Imports

In [ ]:
import os
import sys
from pathlib import Path
import datetime

from ipywidgets import Label
from ipyleaflet  import Map, GeoData, basemaps, LayersControl, ImageOverlay, Marker
from localtileserver import get_leaflet_tile_layer, TileClient

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from file_info import *
from plot import *

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n PrintDate = {} \n brdf_dir = {} \n grid_cell = {} \n index_dir = {} \n home_dir = {}"
      .format(basic_config['today'], basic_config['brdf_dir'],basic_config['grid_cell'],basic_config['index_dir'],basic_config['home_dir']))

%store -r single_plot_params
print("Plotting Parameters: \n plot_yr = {} \n plot_day = {} \n viewband = {} \n image_type = {}"
      .format(single_plot_params['plot_yr'],single_plot_params['plot_day'],single_plot_params['viewband'],single_plot_params['image_type']))

%store -r interactive_plot_params
print( "Shapefile = {} \n If point, file is {} \n If poly, file is {} \n input crs = {}"
     .format (interactive_plot_params['shpfile'], interactive_plot_params['ptfile'],interactive_plot_params['polyfile'],interactive_plot_params['inputCRS']))

In [ ]:
## Get a sample image of type image_type, closest to the date specified with plot_day and plot_yr parameters
if single_plot_params['image_type'] == 'Smooth':
    samp_img = get_closest_image(basic_config['index_dir'],single_plot_params['image_type'],basic_config['data_source'],single_plot_params['plot_yr'],single_plot_params['plot_day'])
else:
    samp_img = get_closest_image(basic_config['brdf_dir'],single_plot_params['image_type'],basic_config['data_source'],single_plot_params['plot_yr'],single_plot_params['plot_day'])

print(samp_img)

## image with color correction, but not interactive

In [ ]:
img = samp_img
img_tif = nc_to_tif(img, out_dir=basic_config['home_dir'])
rgb = get_rbg_img(img_tif,single_plot_params['gamma'])
fig = plt.figure(figsize=(18,12))
plt.imshow(rgb)

## Interactive version

In [ ]:
img = samp_img
img_tif = nc_to_tif(img, out_dir=basic_config['home_dir'])
#m,t = show_interactive_img(img, 5555, out_dir=basic_config['out_dir'])
tile_client = TileClient(img_tif,port=5555)
m = Map(center=tile_client.center(), zoom=12, basemap=basemaps.Esri.WorldImagery)
t = get_leaflet_tile_layer(tile_client, band=[1,2,3])
m.add_layer(t)
m

In [ ]:
#add_shpfile_overlay(shp=interactive_plot_params['shpfile'],ptfile=interactive_plot_params['ptfile'],inputCRS=interactive_plot_params['inputCRS'],polyfile=interactive_plot_params['polyfile'])

In [ ]:
def get_coords(**kwargs):
    if kwargs.get('type') == 'click':
        label = Label()
        label.value = str(kwargs.get('coordinates'))
        coords =eval(label.value) 
        selected_coords.append(coords)
        print(selected_coords)
        return selected_coords

selected_coords = []
m.on_interaction(get_coords)

## Get list of coordinates to use (first click on map above)
#### list is saved as 'out_dir/SelectedCoords.csv'

In [ ]:
coords = convert_and_print_coord_list(selected_coords, img_crs=interactive_plot_params['inputCRS'], out_dir=basic_config['out_dir'])

## Get Values at Coordinates

In [ ]:
get_values_at_coords(coords, interactive_plot_params['inputCRS'], img_tif, [1,2,3])

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html
outName = str(basicConfig['country']+'1c_InteractiveMapSession'
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$outName 1c_ExploreData_InteractiveMap.ipynb